爬取特殊天数 10.31-11.01 11.30-12.1

In [1]:
import time
import pandas as pd
import numpy as np
from xlutils.copy import copy
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

class scrapy_wb():
    def __init__(self,start_time,end_time,book_name_xls,sheet_name_xls,keywords):
        self.year1 = start_time[0] #开始时间
        self.month1 = start_time[1]
        self.day1 = start_time[2]
        self.h1 = start_time[3]
        self.year2 = end_time[0] #结束时间
        self.month2 = end_time[1]
        self.day2 = end_time[2]
        self.h2 = end_time[3]
        self.book_name_xls = book_name_xls #填写你想存放excel的路径，没有文件会自动创建
        self.sheet_name_xls = sheet_name_xls #sheet表名
        self.keywords = keywords #输入你想要的关键字，建议有超话的话加上##，如果结果较少，不加#

    def spider(self):

        # 创建浏览器（浏览器驱动）
        driver = webdriver.Chrome()
        # 打开网页
        url="https://s.weibo.com/weibo?q={}&typeall=1&suball=1&timescope=custom:{}-{:0>2d}-{:0>2d}-{}:{}-{:0>2d}-{:0>2d}-{}&Refer=g&display=0&retcode=6102".format(
            self.keywords, self.year1, self.month1, self.day1, self.h1, self.year2, self.month2, self.day2, self.h2)
        driver.get(url=url)
        # "https://s.weibo.com/weibo?q=%s&typeall=1&suball=1&timescope=custom:%s:%s&Refer=g&display=0&retcode=6102"%(self.keywords,self.start_time,self.end_time)
        # 最大化窗口
        driver.maximize_window()
        # 等待时间是用来自主登录微博账号
        for i in range(20):
            time.sleep(1)
            print('进入等待时间，剩余时间为%s'%(20-i),end='\r',)
        df=pd.DataFrame(np.zeros((1,8),dtype="float"))
        df.columns=["rid", "用户名称", "微博内容", "微博转发量","微博评论量","微博点赞","发布时间","搜索关键词"]
        
        rid = 0

        page=1
        URL1="https://s.weibo.com/weibo?q={}&typeall=1&suball=1&timescope=custom:{}-{:0>2d}-{:0>2d}-{}:{}-{:0>2d}-{:0>2d}-{}&Refer=g&&page={}".format(
        self.keywords, self.year1, self.month1, self.day1, self.h1, self.year2, self.month2, self.day2, self.h2,page)

        driver.get(URL1)
        s=driver.find_elements(by=By.XPATH, value='//div[@class="m-page"]//ul[@class="s-scroll"]//li[last()]')[0].get_attribute('textContent')
        print(s)
        s=s.replace('第','')
        s=s.replace('页','')
        Page=int(s)

        print(self.day1,self.day2,Page)

        for p in range(Page):
            p=p+1
            URL2="https://s.weibo.com/weibo?q={}&typeall=1&suball=1&timescope=custom:{}-{:0>2d}-{:0>2d}-{}:{}-{:0>2d}-{:0>2d}-{}&Refer=g&&page={}".format(
            self.keywords, self.year1, self.month1, self.day1, self.h1, self.year2, self.month2, self.day2, self.h2,p)

            driver.get(URL2)

            elems=driver.find_elements(by=By.XPATH, value='//div[@class="card"]')
            # 一级评论
            for i in range(len(elems)):
                try:
                    rid = rid + 1
                    #用户名
                    weibo_username = elems[0].find_elements(by=By.XPATH, value='//a[@class="name"]')[i].text
                    #微博内容
                    #分为有全文和无全文
                    weibo_content = elems[0].find_elements(by=By.XPATH, value='//p[@class="txt"]')[i].text
                    shares = elems[0].find_elements(by=By.XPATH, value='//div[@class="card-act"]//ul//li[1]')[i].text
                    shares=shares.replace("转发","")
                    comments = elems[0].find_elements(by=By.XPATH, value='//div[@class="card-act"]//ul//li[2]')[i].text
                    comments=comments.replace("评论","")
                    likes = elems[0].find_elements(by=By.XPATH, value='//div[@class="card-act"]//ul//li[3]')[i].text
                    if likes=='赞':
                        likes='0'
                    #发布时间
                    weibo_time = elems[0].find_elements(by=By.XPATH, value='//div[@class="from"]//a[1]')[i].text
                    # print("正在爬取第%s"%rid)

                    df=df.append({"rid":rid, "用户名称":weibo_username, "微博内容":weibo_content, "微博转发量":shares,"微博评论量":comments,"微博点赞":likes,"发布时间":weibo_time,"搜索关键词":self.keywords},ignore_index=True)
                        
                    if float(likes) > 0:
                        button=elems[0].find_elements(by=By.XPATH,value='//div[@class="card-act"]//ul//li[2]//a')[i]
                        # print(button.text)
                        button.click()
                except:
                    pass

            # 二级评论
            txts=driver.find_elements(by=By.XPATH, value='//div[@class="txt"]//a[@class="name"]')
            con=driver.find_elements(by=By.XPATH, value='//div[@class="txt"]')
            t=driver.find_elements(by=By.XPATH, value='//div[@class="fun"]//p')
            # print(len(txts),len(con),len(t))

            for j in range(len(txts)):
                rid = rid + 1
                weibo_username = txts[j].text
                weibo_content=con[j].text.replace(txts[j].text,'')
                weibo_content=weibo_content.replace('：','')
                weibo_time = t[j].text
                df=df.append({"rid":rid, "用户名称":weibo_username, "微博内容":weibo_content, "微博转发量":'',"微博评论量":'',"微博点赞":'',"发布时间":weibo_time,"搜索关键词":'二级评论'},ignore_index=True)
        
        # df=df.append({"rid":'###', "用户名称":'###', "微博内容":'###', "微博转发量":'###',"微博评论量":'###',"微博点赞":'###',"发布时间":'###',"搜索关键词":'###'},ignore_index=True)
        
        df=df.drop(index=0)
        df.to_excel(self.book_name_xls, sheet_name=self.sheet_name_xls)

In [3]:
res=scrapy_wb(start_time = [2022,10,31,14],end_time = [2022,11,1,0],book_name_xls = "F:/Python_code/Play/wb/covidGZwb_10to11.xls",sheet_name_xls = '微博数据',keywords = "广州疫情")
res.spider()

第50页时间，剩余时间为10
31 1 50


C:\Users\wujia\AppData\Local\Temp\ipykernel_12976\309588931.py:111: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(self.book_name_xls, sheet_name=self.sheet_name_xls)


In [2]:
res=scrapy_wb(start_time = [2022,11,30,14],end_time = [2022,12,1,0],book_name_xls = "F:/Python_code/Play/wb/covidGZwb_11to12.xls",sheet_name_xls = '微博数据',keywords = "广州疫情")
res.spider()

第50页时间，剩余时间为10
30 1 50


C:\Users\wujia\AppData\Local\Temp\ipykernel_5876\3467461785.py:112: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(self.book_name_xls, sheet_name=self.sheet_name_xls)


In [3]:
res=scrapy_wb(start_time = [2022,9,30,14],end_time = [2022,10,1,0],book_name_xls = "F:/Python_code/Play/wb/covidGZwb_9to10.xls",sheet_name_xls = '微博数据',keywords = "广州疫情")
res.spider()

第12页时间，剩余时间为10
30 1 12


C:\Users\wujia\AppData\Local\Temp\ipykernel_16524\3467461785.py:112: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(self.book_name_xls, sheet_name=self.sheet_name_xls)
